**About :** Generates candidates.

**TODO**:

In [ ]:
cd ../src

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import os
import gc
import sys
import cudf
import json
import glob
import pickle
import warnings
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from datetime import datetime
from collections import Counter
from pandarallel import pandarallel
from numerize.numerize import numerize


warnings.simplefilter(action="ignore", category=FutureWarning)
pandarallel.initialize(nb_workers=32, progress_bar=False)

In [ ]:
from params import *

from data.covisitation import compute_covisitation_matrix
from data.candidates import load_parquets, create_candidates, explode, matrix_to_candids_dict

from utils.metrics import get_coverage
from utils.chris import suggest_clicks, suggest_buys, read_file_to_cache

## Covisitation matrices
- Recompute on train without using val ??

In [ ]:
MATRIX_FOLDER = "../output/matrices_2/"

os.makedirs(MATRIX_FOLDER, exist_ok=True)

In [ ]:
MODE = "val"

In [ ]:
if MODE == "val":
#     files = glob.glob("../output/full_train_parquet/*") +  glob.glob("../output/val_parquet/*")
    files = glob.glob("../output/full_train_2_parquet/*") +  glob.glob("../output/val_2_parquet/*")
elif MODE == "test":
    files = glob.glob("../output/full_train_val_parquet/*") +  glob.glob("../output/test_parquet/*")
else:
    raise NotImplementedError

In [ ]:
data_cache = {}
for f in tqdm(files):
    data_cache[f] = read_file_to_cache(f)

In [ ]:
for n in [20, 40]:
    compute_covisitation_matrix(
        files,
        data_cache,
        weighting="temporal",
        n=n,
        save_folder=MATRIX_FOLDER,
        suffix=MODE,
    )

    compute_covisitation_matrix(
        files,
        data_cache,
        weighting="type",
        type_weight={0: 1, 1: 3, 2: 6},
        n=n,
        save_folder=MATRIX_FOLDER,
        suffix=MODE,
    )
    
    compute_covisitation_matrix(
        files,
        data_cache,
        considered_types=[1, 2],
        weighting="",
        n=n,
        save_folder=MATRIX_FOLDER,
        suffix=MODE,
    )

    compute_covisitation_matrix(
        files,
        data_cache,
        weighting="type",
        type_weight={0: 0.5, 1: 9, 2: 0.5},
        n=n,
        save_folder=MATRIX_FOLDER,
        suffix=MODE,
    )

Done